# Estimate wind speeds from flight data



In [1]:
from flightanalysis import Section
import plotly.express as px
from geometry import Points
from geometry import Point


#    Section.from_csv("examples/data/P23.csv"),
#    Section.from_csv("examples/data/loops.csv"),


In [2]:
from flightanalysis import Line
from geometry import Transformation, Quaternion
import numpy as np

#create a straight line, upright at 30m/s, along the X axis
judge = Line(100).create_template(Transformation(
    Point(0, 0,0), Quaternion.from_euler((np.pi,0,0))
), 30.0)


#simulate some alpha
body = judge.judging_to_wind(Points.Z(np.full(len(judge.data), -1.0)))

#create a constant wind field, 20m/s in the Y direction
env_wind = Points.Y(np.full(len(judge.data), 10.0))

#convert the section to wind axis
wind = judge.judging_to_wind(env_wind)
body = body.judging_to_wind(env_wind)

In [3]:
body = Section.from_csv("examples/data/P23.csv").flying_only()
judge = body.to_judging()



In [4]:
from scipy.optimize import minimize
from flightanalysis.environment.wind import get_wind_error, wind_power_law_builder

cost_fn = lambda args : abs(get_wind_error(args, wind_power_law_builder, body, judge))

res = minimize(cost_fn, [0.0, 3.0, 0.2], method="Powell", bounds= [(0.0, 2 * np.pi), (0.05, 20.0), (0.1, 0.4)])

wind_model = wind_power_law_builder(res.x)

res


   direc: array([[ 0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 1.55873348,  3.3004203 , -0.01615484]])
     fun: 192251.28765416253
 message: 'Optimization terminated successfully.'
    nfev: 172
     nit: 3
  status: 0
 success: True
       x: array([ 1.97186344, 10.04353633,  0.12347796])

In [13]:
import pandas as pd
from itertools import product
heads = np.linspace(0.0, np.pi * 2, 20)
speeds = np.linspace(0.1, 5.0, 20)
ais = np.array([0.1,0.2,0.4])

vars = list(product(heads, speeds, ais))
ress = [cost_fn(var) for var in vars]

df = pd.DataFrame(
    np.concatenate([np.array(vars), np.array(ress).reshape(-1,1)], axis=1),
    columns=["heading", "speed", "a", "cost"]
)



In [14]:
px.scatter(df.loc[df.a==0.2], x="heading", y="cost", color="speed")


In [19]:
from flightanalysis.model.parameters import ACConstants, cold_draft
from flightanalysis import get_q


wind_model = wind_power_law_builder([5.952491, 4.7421, 0.2])
#wind_model = wind_power_law_builder(res.x)
wind_vectors =1* wind_model(body.gpos.z, np.zeros(len(body.data)))

wind = judge.judging_to_wind(wind_vectors)

alpha, beta = body.measure_aoa(wind)

airspeed = wind.measure_airspeed(wind_vectors)

wind_force_coeff = wind.measure_coefficients(cold_draft.mass, get_q(1.225, airspeed.x), cold_draft.s )


In [24]:


df = pd.DataFrame(
    np.stack([np.degrees(alpha), wind_force_coeff.z], axis=1), 
    columns=["alpha", "cz"]
)

fitdata = df.loc[np.abs(df.alpha) < 8 ].loc[np.abs(df.cz)<5]

fit = np.polyfit(fitdata.alpha,fitdata.cz, deg=1)

x = np.linspace(-10, 10, 10)
y=fit[1] + fit[0] * x

def error(x,y):
    return np.abs(y - fit[1] - fit[0] * x)

print(np.sum(error(df.alpha, df.cz)))

fig = go.Figure()

dfs2 = df.loc[np.abs(df.alpha) < 15].loc[np.abs(df.cz)<3]

fig.add_trace(go.Scatter(x=dfs2.alpha, y=dfs2.cz, mode="markers"))

fig.add_trace(go.Scatter(x=x, y=y))
fig.update_layout(width=800, height=800)

#x,y


968940.8425056075


In [10]:
from flightplotting.plots import plotsec

start = 100
end=105
fig = plotsec(body.subset(start,end), nmodels=5, scale=20)

fig = plotsec(judge.subset(start,end), nmodels=5, scale=20, color="blue", fig=fig)
fig = plotsec(wind.subset(start,end), nmodels=5, scale=20, color="red", fig=fig)

#fig.update_layout(width=1200, height=1200)

In [11]:



a_err = 3
clerr = 0.3

def noisy_lift(a_err, clerr, npo=(10, 30, 10)):

    x=np.linspace(-20,20,np.sum(npo)) +(np.random.random(np.sum(npo)) - 0.5) * a_err
    y = np.concatenate([
        np.full(npo[0],-1)
        ,np.linspace(-1,1, npo[1]),
        np.full(npo[2],1)
    ]) + np.random.random(np.sum(npo))*clerr
    return dict(x=x, y=y, a_err=a_err, clerr=clerr)


nlifts = [noisy_lift(aerr, 0.3) for aerr in range(0, 20, 5) ]


fig=go.Figure()

for nlift in nlifts:
    fig.add_trace(go.Scatter(x=nlift['x'], y=nlift['y'], name=nlift['a_err'], mode="markers"))

fig